In [39]:
import numpy as np
import pandas as pd
import csv
import os
import re
import time
from pathlib import Path
import ffmpeg
from importlib.metadata import version
import logging

In [40]:
# Set logging level to DEBUG
logging.basicConfig(level=logging.DEBUG)

# Replace 'video.mp4' with the path to your media file
probe = ffmpeg.probe('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/Camera/GUT/S01/C02/Untitled 140.mp4')
print(probe)

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'Main', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 640, 'height': 480, 'coded_width': 640, 'coded_height': 480, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '4:3', 'pix_fmt': 'yuv420p', 'level': 30, 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '33/1', 'avg_frame_rate': '1853056/68201', 'time_base': '1/15360', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 8184120, 'duration': '532.820312', 'bit_rate': '103042', 'bits_per_raw_sample': '8', 'nb_frames': '14477', 'extradata_size': 39, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, '

In [2]:
version('ffmpeg-python')

'0.2.0'

In [3]:
PYCHARM_DEBUG=True

In [41]:
def print_table(table):
    for row in table:
        print(row)

def create_table(tab1, tab2, tab3):
    result_tab = []

    emotions = ['Unknown', 'Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']

    for row1, row2, row3 in zip(tab1, tab2, tab3):
        sum_rows = [int(a) + int(b) + int(c) for a, b, c in zip(row1[1:], row2[1:], row3[1:])]

        total_sum = sum(sum_rows)
        if total_sum > 0:
            # Calculate the percentage distribution
            percentages = [round((x / total_sum) * 100, 2) for x in sum_rows]
        else:
            percentages = [0] * len(sum_rows)

        result_tab.append([row1[0]] + percentages)

    return result_tab

# Makes table equal
def fill_table(table, csvreader):
    seconds = 0
    stop = 0
    for row in csvreader:
        row[0] = int(row[0].split('.')[0])
        if seconds <= row[0] and stop == 0:  #fill table with missing seconds
            for i in range(0, row[0]):
                table.append([i, '0', '0', '0', '0', '0', '0', '0'])
                seconds += 1
            stop = 1
            table.append(row)
        else:
            table.append(row)


# Create percentages for every second
def csv_files_reader(base_path):
    index = ['I', 'II', 'III']
    tab1 = []
    tab2 = []
    tab3 = []

    if os.path.exists(base_path):

        for i in index:
            file = os.path.join(base_path, i)
            for file_name in os.listdir(file):
                file_path = os.path.join(file, file_name)

                with open(file_path, 'r') as file:
                    csvreader = csv.reader(file)
                    header = next(csvreader)

                    if i == 'I':
                        fill_table(tab1, csvreader)
                    elif i == 'II':
                        fill_table(tab2, csvreader)
                    elif i == 'III':
                        fill_table(tab3, csvreader)
    else:
        print(f"File path {base_path} doesn't exist.")

    large_table = max([tab1, tab2, tab3], key=len)

    for tab in [tab1, tab2, tab3]:
        if len(tab) < len(large_table):
            diff = len(large_table) - len(tab)

            if len(tab) == 0:
                sec = 0
                for sec in range(len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])
            else:
                sec = tab[-1][0]  #latest second in table
                for sec in range(tab[-1][0] + 1, len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])

    return create_table(tab1, tab2, tab3)


def get_boris_vector(path, target):
    table = csv_files_reader(path)
    df = pd.DataFrame(table)
    df.to_csv(target, index=False)

In [100]:
def get_video_metadata(file_path):
    try:
        # Get the file status
        file_stat = os.stat(file_path)

        # Get the last modified time
        last_modified_time_str = time.ctime(file_stat.st_mtime)
        print(last_modified_time_str)
        last_modified_timestamp = int(file_stat.st_mtime)
        print(last_modified_timestamp)
        # Get the duration of the video
        probe = ffmpeg.probe(file_path)
        duration = float(probe['format']['duration'])

        metadata = {
            'last_modified_time': last_modified_time_str,
            'last_modified_timestamp': last_modified_timestamp,
            'duration': int(duration),  # Duration in seconds
            'initial_timestamp' : last_modified_timestamp - int(duration)
        }
        print(metadata)
        return metadata

    except FileNotFoundError:
        return f"The file {file_path} does not exist."
    except Exception as e:
        return f"An error occurred: {str(e)}"
    

In [101]:
# Get the moment of the start of the vector and its frequency
def get_unix_and_hz(file_path):
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        unix = int(float(next(reader)[0]))  # Convert the first cell to integer
        hz = int(float(next(reader)[0]))  # Convert the second cell to integer
    return unix, hz

In [102]:
# Trim the vector to be the multiple of its frequency
def trim_vector(vector, rate):
    length = len(vector)
    if length % rate != 0:
        # Calculate how many elements need to be removed
        excess_elements = length % rate
        # Trim the vector
        vector = vector[:-excess_elements]
    return vector

In [103]:
# Change the frequency by averaging the values
def mean_of_values(vector, rate):
    # Ensure the length of the vector is a multiple of n
    if len(vector) % rate != 0:
        raise ValueError("Length of the vector must be a multiple of frequency")

    # Reshape the vector into a 2D array where each row is a group of n elements
    reshaped_vector = np.reshape(vector, (-1, rate))

    # Calculate the mean along the rows
    mean_values = np.mean(reshaped_vector, axis=1)

    return mean_values

In [116]:
def slice_vectors(video_path, biosignal_path, signals_storage_path, video_storage_path):
    
    # Get the video metadata
    video = get_video_metadata(video_path)
    print(video)
    # Get the path for EDA, TEMP and HR
    EDA_path = os.path.join(biosignal_path, 'EDA.csv')
    TEMP_path = os.path.join(biosignal_path, 'TEMP.csv')
    HR_path = os.path.join(biosignal_path, 'HR.csv')

    # Get only the data
    EDA = pd.read_csv(EDA_path, skiprows = 2, header = None)
    TEMP = pd.read_csv(TEMP_path, skiprows = 2, header = None)
    HR = pd.read_csv(HR_path, skiprows = 2, header = None)

    # Get the starting time and frequency
    unix_EDA, hz_EDA = get_unix_and_hz(EDA_path)
    unix_TEMP, hz_TEMP = get_unix_and_hz(TEMP_path)
    unix_HR, hz_HR = get_unix_and_hz(HR_path)
    unix_video = video['initial_timestamp']

    # Unificate the frequencies  
    EDA_mean = mean_of_values(trim_vector(EDA, hz_EDA), hz_EDA)
    TEMP_mean = mean_of_values(trim_vector(TEMP, hz_TEMP), hz_TEMP)
    HR_mean = mean_of_values(trim_vector(HR, hz_HR), hz_HR)

    # Get the lengths
    length_EDA = len(EDA_mean)
    length_TEMP = len(TEMP_mean)
    length_HR = len(HR_mean)
    length_video = video['duration']
    
    # Get the vectors of starts and ends for biosignals and video
    starts = [unix_HR, unix_TEMP, unix_EDA, unix_video]
    ends = [unix_HR + length_HR, unix_TEMP + length_TEMP, unix_EDA + length_EDA, unix_video + length_video]

    # Get the latest start of any vector
    last_start = max(starts)
    
    # Get the earliest end of any vector
    first_end = min(ends)

    # Get matching indexes for start and end for every vector
    EDA_first_index = last_start - unix_EDA
    EDA_last_index = first_end - unix_EDA
    TEMP_first_index = last_start - unix_TEMP
    TEMP_last_index = first_end - unix_TEMP
    HR_first_index = last_start - unix_HR
    HR_last_index = first_end - unix_HR
    video_first_index = last_start - unix_video
    video_last_index = first_end - unix_video


    print('EDA: ', EDA_last_index - EDA_first_index)
    print('TEMP: ', TEMP_last_index - TEMP_first_index)
    print('HR: ', HR_last_index - HR_first_index)
    print('video: ', video_last_index - video_first_index)
    
    # Slice the biosignals based on the index
    sliced_EDA = EDA[EDA_first_index:EDA_last_index]
    sliced_TEMP = TEMP[TEMP_first_index:TEMP_last_index]
    sliced_HR = HR[HR_first_index:HR_last_index]
    
    # Create one df with all biosignals
    biosignals = pd.concat([sliced_EDA.reset_index(drop=True), sliced_TEMP.reset_index(drop=True), sliced_HR.reset_index(drop=True)], axis=1)
    
    # Rename column names
    biosignals.columns = ['EDA', 'TEMP', 'HR']
    
    # Cut the video based on the index and add a file
    #ffmpeg_extract_subclip(video_path, video_first_index, video_last_index, targetname=video_storage_path)
    ffmpeg.input(str(video_path)).output(str(video_storage_path), ss=video_first_index, t=video_last_index-video_first_index, codec='copy').run()

    # Add the biosignals to file
    biosignals.to_csv(signals_storage_path, index=False)

In [117]:
def traverse():
    root_dir = '//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE'
    target_root_dir = '//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis'
    boris_ext = 'Analysis/BORIS/'
    type = ['Camera', 'Wristband']
    research_centers = ['GUT']
    #research_centers = ['GUT', 'ITU-YU', 'MAAP']
    s_values = ["S01"]
    #s_values = ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11" ]
    c_values = ["C02"]
    #c_values = ["C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13" ]
    for research_center in research_centers:
        for session in s_values:
            # Create the session path for biosignal and camera + BORIS
            camera_path = Path(root_dir).joinpath(type[0], research_center, session)
            signals_path = Path(root_dir).joinpath(type[1], research_center, session)
            if camera_path.is_dir() and signals_path.is_dir():
                # Create the meeting path for biosignal and camera + BORIS
                for camera in c_values:
                    exact_camera_path = Path(camera_path).joinpath(camera)
                    exact_signals_path = Path(signals_path).joinpath(camera)
                    if exact_camera_path.is_dir() and exact_signals_path.is_dir():
                        boris_path = Path(exact_camera_path).joinpath(boris_ext)
                        if boris_path.is_dir():
                            # Quick fix of two MAAP sessions being divided TODO: Fix this
                            if not(research_center == 'MAAP' and ((session == 'S01' and camera == 'C05') or (session == 'S03' and camera == 'C05'))):
                                # Find exact video name
                                video_ext = '_video.mp4'
                                if research_center == 'GUT':
                                    mp4_pattern = re.compile(r'^Untitled \d+\.mp4$')
                                elif research_center == 'ITU-YU':
                                    mp4_pattern = re.compile(r'^ITU-C\d{2}-S\d{2}-\d{8}-Camera\.mp4$')
                                elif research_center == 'MAAP':
                                    mp4_pattern = re.compile(r'^MAAP-C\d{2}-S\d{2}-\d{8}-Camera\.AVI$')
                                    video_ext = '_video.AVI'
                                for files_in_camera_dir in os.listdir(exact_camera_path):
                                    if mp4_pattern.match(files_in_camera_dir):
                                        video_path = os.path.join(exact_camera_path, files_in_camera_dir)
                                        print(video_path)
                                
                                filename = research_center + '_' + session + '_' + camera
                                boris_filename = filename + 'BORIS.csv'
                                boris_target_dir = Path(target_root_dir).joinpath(research_center, boris_filename)
                                
                                signal_filename = filename + '_biosignals.csv'
                                signal_target_dir = Path(target_root_dir).joinpath(research_center, signal_filename)
                                
                                
                                video_filename = filename + video_ext
                                video_target_dir = Path(target_root_dir).joinpath(research_center, video_filename)
                                
                                # Checking if everything is fine
                                print(f"{video_target_dir} exists.")
                                
                                # Create combined BORIS vector
                                #get_boris_vector(boris_path, boris_target_dir)
                                
                                # Create one vector of biosignals, sliced BORIS and sliced video
                                slice_vectors(video_path, exact_signals_path, signal_target_dir, video_target_dir)
                                
                                
                                
                                # TODO: sprawdz dlaczego wektory mają różne długości 
traverse()

\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S01\C02\Untitled 140.mp4
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\de-earlyfusionthesis\GUT\GUT_S01_C02_video.mp4 exists.
Fri Jun 18 11:34:18 2021
1624008858
{'last_modified_time': 'Fri Jun 18 11:34:18 2021', 'last_modified_timestamp': 1624008858, 'duration': 532, 'initial_timestamp': 1624008326}
{'last_modified_time': 'Fri Jun 18 11:34:18 2021', 'last_modified_timestamp': 1624008858, 'duration': 532, 'initial_timestamp': 1624008326}
EDA:  462
TEMP:  462
HR:  462
video:  462


In [115]:
main_directory = 'C:/Users/oskik/PycharmProjects/InżynierkaSandbox/'
s01_values = ["S01_boris/", "S01_wristband/"]
c_values = ["C01/", "C02/", "C03/"]
mp3_pattern = re.compile(r'Untitled \d+\.mp4')

for c_value in c_values:
    for s01_value in s01_values:
        if s01_value == "S01_boris/":
            movie_dir_path = os.path.join(main_directory, s01_value, c_value)
            print(movie_dir_path)
            if os.path.isdir(movie_dir_path):
                for file_name in os.listdir(movie_dir_path):
                    print(file_name)
                    if mp3_pattern.match(file_name):
                        movie_path = os.path.join(main_directory, s01_value, c_value, file_name)
                        print(movie_path)

            video = get_video_metadata(movie_path)
            unix_video = video['initial_timestamp']

        elif s01_value == "S01_wristband/":
            main_path = os.path.join(main_directory, s01_value, c_value)

            EDA_path = os.path.join(main_path, 'EDA.csv')
            TEMP_path = os.path.join(main_path, 'TEMP.csv')
            HR_path = os.path.join(main_path, 'HR.csv')

            EDA = pd.read_csv(EDA_path, skiprows = 2, header = None)
            TEMP = pd.read_csv(TEMP_path, skiprows = 2, header = None)
            HR = pd.read_csv(HR_path, skiprows = 2, header = None)

            unix_EDA, hz_EDA = get_unix_and_hz(EDA_path)
            unix_TEMP, hz_TEMP = get_unix_and_hz(TEMP_path)
            unix_HR, hz_HR = get_unix_and_hz(HR_path)

            EDA_mean = mean_of_values(trim_vector(EDA, hz_EDA), hz_EDA)
            TEMP_mean = mean_of_values(trim_vector(TEMP, hz_TEMP), hz_TEMP)
            HR_mean = mean_of_values(trim_vector(HR, hz_HR), hz_HR)

            length_EDA = len(EDA_mean)
            length_TEMP = len(TEMP_mean)
            length_HR = len(HR_mean)

    starts = [unix_HR, unix_TEMP, unix_EDA, unix_video]
    ends = [unix_HR + length_HR, unix_TEMP + length_TEMP, unix_EDA + length_EDA, unix_video + video['duration']]

    last_start = max(starts)
    first_end = min(ends)

    EDA_first_index = last_start - unix_EDA
    EDA_last_index = first_end - unix_EDA
    TEMP_first_index = last_start - unix_TEMP
    TEMP_last_index = first_end - unix_TEMP
    HR_first_index = last_start - unix_HR
    HR_last_index = first_end - unix_HR
    video_first_index = last_start - unix_video
    video_last_index = first_end - unix_video

    sliced_EDA = EDA[EDA_first_index:EDA_last_index]
    sliced_TEMP = TEMP[TEMP_first_index:TEMP_last_index]
    sliced_HR = HR[HR_first_index:HR_last_index]

    ffmpeg_extract_subclip(movie_path, video_first_index, video_last_index, targetname=movie_dir_path+'sliced.mp4')
    ffmpeg.input(input_file, ss=4, t=49).output(output_file, codec='copy').run()

    sliced_EDA_path = main_path+'sliced_EDA.csv'
    sliced_TEMP_path = os.path.join(main_path, 'sliced_TEMP.csv')
    sliced_HR_path = os.path.join(main_path, 'sliced_HR.csv')

    sliced_EDA.to_csv(sliced_EDA_path, index=False)
    sliced_TEMP.to_csv(sliced_TEMP_path, index=False)
    sliced_HR.to_csv(sliced_HR_path, index=False)

C:/Users/oskik/PycharmProjects/InżynierkaSandbox/S01_boris/C01/
00219.MTS
Analysis
sliced.mp4
Thumbs.db
Untitled 139.mp4
C:/Users/oskik/PycharmProjects/InżynierkaSandbox/S01_boris/C01/Untitled 139.mp4
Fri Jun 18 11:15:24 2021
1624007724
{'last_modified_time': 'Fri Jun 18 11:15:24 2021', 'last_modified_timestamp': 1624007724, 'duration': 609, 'initial_timestamp': 1624007115}


NameError: name 'ffmpeg_extract_subclip' is not defined